In [15]:
import pandas as pd

# 1. Leer los archivos CSV
inv_df = pd.read_csv("Data/Inv.csv", sep=",")
offline_df = pd.read_csv("Data/Offline.csv", sep=",")
time_df = pd.read_csv("Data/Time.csv", sep=",")
visit_df = pd.read_csv("Data/Visit.csv", sep=",")
web_df = pd.read_csv("Data/Web.csv", sep=",")

# 2. Asegurarse de que la columna ID_Date es tipo string para merge correcto
for df in [inv_df, offline_df, time_df, visit_df, web_df]:
    df["ID_Date"] = df["ID_Date"].astype(str)

# 3. Unir todos los datasets por ID_Date
df_combined = web_df \
    .merge(visit_df, on="ID_Date", how="left") \
    .merge(offline_df, on="ID_Date", how="left") \
    .merge(time_df, on="ID_Date", how="left") \
    .merge(inv_df, on="ID_Date", how="left")

# 4. Crear columna Fecha real a partir de ID_Date
df_combined["Fecha"] = pd.to_datetime(df_combined["ID_Date"] + "01", format="%Y%m%d")

# 5. Crear columnas de eventos con valor por defecto 0
df_combined["Navidad"] = 0
df_combined["Black_Friday"] = 0
df_combined["Semana_Santa"] = 0

# 6. Marcar NAVIDAD (20 dic al 8 ene)
df_combined["Navidad"] = df_combined["Fecha"].apply(
    lambda x: 1 if (x.month == 12 and x.day >= 20) or (x.month == 1 and x.day <= 8) else 0
)

# 7. Marcar BLACK FRIDAY (último viernes de noviembre)
def es_black_friday(fecha):
    if fecha.month == 11:
        last_friday = pd.Timestamp(year=fecha.year, month=11, day=30)
        while last_friday.weekday() != 4:  # 4 = viernes
            last_friday -= pd.Timedelta(days=1)
        return int(fecha.date() == last_friday.date())
    return 0

df_combined["Black_Friday"] = df_combined["Fecha"].apply(es_black_friday)

# 8. Marcar SEMANA SANTA (20 marzo al 15 abril aprox)
df_combined["Semana_Santa"] = df_combined["Fecha"].apply(
    lambda x: 1 if (x.month == 3 and x.day >= 20) or (x.month == 4 and x.day <= 15) else 0
)

# 9. (Opcional) Guardar el resultado como CSV
df_combined.to_csv("Data/df_con_eventos.csv", index=False)

print("✅ Dataset combinado y eventos añadidos correctamente.")



✅ Dataset combinado y eventos añadidos correctamente.


In [16]:
df_combined

,ID_Date,Unique_visitors,PDFBrochuresDownloaded,ProductConfigurator,Product_configurator_Visists,SocialNetworks,DirectTraffic,EMail,NaturalSearch,OnlineMedia,...,PRODUCCION,RADIO,REVISTAS,PlataformasVideo,VARIOS,INV_Total,Fecha,Navidad,Black_Friday,Semana_Santa
0,202201,272382,4838,33222,94248,1415,65373,25,167199,63649,...,6050,"2426,4","18013,5","338824,94","7531,030000000001","574600,14",2022-01-01,1,0,0
1,202202,352581,5823,38570,109531,546,77209,26,176733,124861,...,6700,0,"49110,909999999996","487537,14","6972,02","916515,0800000001",2022-02-01,0,0,0
2,202203,332955,4845,33956,97805,5444,83178,341,173514,104132,...,3700,0,0,75000,"15284,85","367522,47",2022-03-01,0,0,0
3,202204,282169,3711,45949,90689,1184,69335,162,162805,74044,...,21550,1140,40876,"610865,8899999999","12255,75","941187,9099999999",2022-04-01,0,0,1
4,202205,261829,9037,48685,92069,8648,70972,210,156011,44196,...,4900,"102851,86000000002","20775,75","393807,7400000001","7926,459999999999","746390,17",2022-05-01,0,0,0
5,202206,201141,10727,38608,73421,1082,61455,2374,134355,15600,...,12250,"5526,49","75573,13",0,"5405,719999999999","203983,00000000003",2022-06-01,0,0,0
6,202207,206538,11023,38912,74564,909,66835,204,144300,11000,...,650,"158,4",2600,1500,"4574,5","130196,56999999999",2022-07-01,0,0,0
7,202208,259903,15743,47613,88335,7511,78704,14,172986,15205,...,9750,"345,6",33860,"391987,8699999999","28716,26","524021,7399999999",2022-08-01,0,0,0
8,202209,261618,14130,44164,83840,1773,58630,199,174125,41832,...,7850,"2686,8",14580,0,"6645,2699999999995","158223,58",2022-09-01,0,0,0
9,202210,396642,16321,53200,100262,2117,74040,874,203699,162588,...,20300,3162,"117786,97","804181,4000000003","14421,129999999997","1277335,3600000003",2022-10-01,0,0,0


In [17]:
# Reemplazar comas por puntos y convertir a numérico donde sea necesario
for col in df_combined.columns:
    if df_combined[col].dtype == 'object':
        df_combined[col] = df_combined[col].str.replace(',', '.', regex=False)
        df_combined[col] = pd.to_numeric(df_combined[col], errors='ignore')  # Solo convierte si se puede


C:\Users\aserr\AppData\Local\Temp\ipykernel_31200\700684367.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_combined[col] = pd.to_numeric(df_combined[col], errors='ignore')  # Solo convierte si se puede


In [18]:
# Asegúrate de que las columnas clave sean numéricas
df_combined['Sales'] = pd.to_numeric(df_combined['Sales'], errors='coerce')
df_combined['Unique_visitors'] = pd.to_numeric(df_combined['Unique_visitors'], errors='coerce')

# Crear ratios
df_combined['Ratio_Ventas_Trafico'] = df_combined['Sales'] / df_combined['Unique_visitors']
df_combined['Ratio_Ventas_Trafico_Lag1'] = df_combined['Ratio_Ventas_Trafico'].shift(1)

# Otros ratios si los quieres
df_combined['Ventas_Lag1'] = df_combined['Sales'].shift(1)
df_combined['Crecimiento_Ventas'] = (df_combined['Sales'] - df_combined['Ventas_Lag1']) / df_combined['Ventas_Lag1']

df_combined['Trafico_Lag1'] = df_combined['Unique_visitors'].shift(1)
df_combined['Crecimiento_Trafico'] = (df_combined['Unique_visitors'] - df_combined['Trafico_Lag1']) / df_combined['Trafico_Lag1']

# Eliminar filas con NaN (por los lags)
df_combined.dropna(inplace=True)


In [19]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Asegúrate de que estos nombres de columnas coincidan con los de tu dataset
X_vars = [
    'PRENSA', 'PRODUCCION', 'RADIO', 'PlataformasVideo', 'REVISTAS', 'VARIOS',
    'PaidSearch', 'SocialNetworks', 'NaturalSearch', 'Unique_visitors',
    'Navidad', 'Black_Friday', 'Semana_Santa',
    'Ratio_Ventas_Trafico_Lag1'
]

# Confirmamos que todo está en tipo numérico
for col in X_vars + ['Sales']:
    df_combined[col] = pd.to_numeric(df_combined[col], errors='coerce')

X = df_combined[X_vars]
y = df_combined['Sales']

# Estandarizamos las variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ridge Regression
ridge = Ridge(alpha=1.0)
ridge.fit(X_scaled, y)

# Coeficientes
coef_df = pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': ridge.coef_
}).sort_values(by='Coeficiente', ascending=False)

print("Coeficientes del modelo Ridge:")
print(coef_df)


Coeficientes del modelo Ridge:
                     Variable  Coeficiente
9             Unique_visitors   156.998414
3            PlataformasVideo    45.031939
13  Ratio_Ventas_Trafico_Lag1    22.676780
1                  PRODUCCION    19.071670
0                      PRENSA    14.817056
7              SocialNetworks    13.709215
11               Black_Friday     0.000000
2                       RADIO   -11.078839
6                  PaidSearch   -24.200607
4                    REVISTAS   -67.222531
10                    Navidad   -74.891406
12               Semana_Santa  -113.436876
5                      VARIOS  -125.457302
8               NaturalSearch  -147.456702
